## Свой numpy

код написан с рекомендациями pep

In [1]:
import numpy as np

In [2]:
# pip install pycodestyle flake8 pycodestyle_magic

In [3]:
%load_ext pycodestyle_magic

In [4]:
%%pycodestyle

a = 1 '# eee'
s = 2

In [170]:
# %%pycodestyle

class Array(list):
    """Сложение, когда аргумент слева"""
    def __add__(self, other):
        if isinstance(other, (int, float)):
            return Array([x + other for x in self])
        elif isinstance(other, (Array, list, tuple)):
            if len(self) != len(other):
                raise ValueError('Сложение векторов разной длины недопустимо')
            else:
                return Array([x + y for x, y in zip(self, other)])
        else:
            raise TypeError(f"Вектор нельзя сложить с {other}")
    """Сложение, когда аргумент справа"""
    def __radd__(self, other):
        return self + other

    """Вычитание, когда аргумент слева"""
    def __sub__(self, other):
        return self.sub(other, 'left')
    """Вычитание, когда аргумент справа"""
    def __rsub__(self, other):
        return self.sub(other, 'right')

    def sub(self, other, flag):
        if isinstance(other, (int, float)):
            if flag == 'left':
                return Array([x - other for x in self])
            elif flag == 'right':
                return Array([other - x for x in self])
        elif isinstance(other, (Array, list, tuple)):
            if len(self) != len(other):
                raise ValueError('Вычитание векторов разной длины недопустимо')
            else:
                return Array([x - y for x, y in zip(self, other)])
        else:
            raise TypeError(f"Вектор нельзя вычисть с {other}")

    """Умножение, когда аргумент слева"""
    def __mul__(self, other):
        if isinstance(other, (int, float)):
            return Array([x * other for x in self])
        elif isinstance(other, (Array, list, tuple)):
            if len(self) != len(other):
                raise ValueError('Умножение векторов разной длины недопустимо')
            else:
                return Array([x * y for x, y in zip(self, other)])
        else:
            raise TypeError(f"Вектор нельзя умножить с {other}")
    """Умножение, когда аргумент справа"""
    def __rmul__(self, other):
        return self * other

    """Деление, когда аргумент слева"""
    def __truediv__(self, other):
        return self.truediv(other, 'left')
    """Деление, когда аргумент справа"""
    def __rtruediv__(self, other):
        return self.truediv(other, 'right')

    def truediv(self, other, flag):
        if isinstance(other, (int, float)):
            if flag == 'left':
                return Array([x / other for x in self])
            elif flag == 'right':
                return Array([other / x for x in self])
        elif isinstance(other, (Array, list, tuple)):
            if len(self) != len(other):
                raise ValueError('Деление векторов разной длины недопустимо')
            else:
                return Array([x / y for x, y in zip(self, other)])
        else:
            raise TypeError(f"Вектор нельзя разделить с {other}")
            
    """Добавление функционала слайсинга"""
    def __getitem__(self, item):
        if type(item) != tuple:
            item = super().__getitem__(item)
            if isinstance(item, (float, int)):
                return item
            return Array(item)
        else:
            if type(item[0]) == slice:
                return Array([i[item[1]] for i in self[item[0]]])
            else:
                while len(item) > 0:
                    if isinstance(self, (int, float)):
                        raise IndexError('Слишком много индексов для массива этого пространства')
                    self = self[item[0]]
                    item = item[1:]
                return self

    """Находит min число в Array"""
    def min(self):
        while not isinstance(self, (int, float)):
            self = min(self)
        return self
    """Находит max число в Array"""
    def max(self):
        while not isinstance(self, (int, float)):
            self = max(self)
        return self
    """Считает сумму всех чисел в Array"""
    def sum(self):
        sums = 0
        for i in self:
            if isinstance(i, (int, float)):
                sums += i
            else:
                sums += summ(i)
        return sums
    """Считает кол-во всех элементов в Array"""
    def count(self):
        counts = 0
        for i in self:
            if isinstance(i[0], (int, float)):
                counts += len(i)
            else:
                counts += count(i)
        return counts
    """Считает сумму и кол-во всех элементов в Array"""
    def sum_count(self):
        counts = 0
        sums = 0
        for i in self:
            if isinstance(i, (int, float)):
                counts += 1
                sums += i
            else:
                s, c = Array(i).sum_count()
                sums += s
                counts += c
                
        return sums, counts
    """Считает среднее значение элементов в Array"""
    def mean(self):
        s, c = self.sum_count()
        return s / c

## Тестирование mean, max, count...

In [171]:
list1 = [[1,2,3], [4,5,6], [7,8,9]]
a = Array(list1)
b = np.array(list1)

In [172]:
a.min() == b.min()

True

In [173]:
a.max() == b.max()

True

In [174]:
b.mean() == a.mean()

True

In [175]:
a.sum() == b.sum()

True

In [176]:
a.sum_count()

(45, 9)

In [179]:
a.count()

9

## Тестирование арифметики

In [6]:
def add(x, y):
    return x + y

def sub(x, y):
    return x - y

def mul(x, y):
    return x * y

def div(x, y):
    return x / y

In [7]:
a = Array

In [8]:
list1 = [1,2,3]
list2 = [4,5,6]
a = Array(list1)
a2 = np.array(list1)

b = Array(list2)
b2 = np.array(list2)

cool = True
for en, fun in enumerate([add, sub, mul, div]):
    if all(fun(a, b) == fun(a2, b2)): 
        print(f"{en} Ok - a {str(fun)[10:13]} b")
    else:
        cool = False
        print(f"{en} Ko - a {str(fun)[10:13]} b")
        
    if all(fun(b, a) == fun(b2, a2)):
        print(f"{en} Ok - b {str(fun)[10:13]} a\n")
    else:
        cool = False
        print(f"{en } Ko - b {str(fun)[10:13]} a\n")
        
    if all(fun(a, 3) == fun(a2, 3)):
        print(f"{en} Ok - a {str(fun)[10:13]} 3")
    else:
        cool = False
        print(f"{en} Ko - a {str(fun)[10:13]} 3")
        
    if all(fun(3, a) == fun(3, a2)):
        print(f"{en} Ok - 3 {str(fun)[10:13]} a\n")
    else:
        cool = False
        print(f"{en } Ko - 3 {str(fun)[10:13]} a\n")
    
print('--------------------')
if cool:
    print('Ошибок нет')
else:
    print('Ошибка!!!!!')

0 Ok - a add b
0 Ok - b add a

0 Ok - a add 3
0 Ok - 3 add a

1 Ok - a sub b
1 Ok - b sub a

1 Ok - a sub 3
1 Ok - 3 sub a

2 Ok - a mul b
2 Ok - b mul a

2 Ok - a mul 3
2 Ok - 3 mul a

3 Ok - a div b
3 Ok - b div a

3 Ok - a div 3
3 Ok - 3 div a

--------------------
Ошибок нет


## Тестирование слайсинга

In [9]:
def slice_intepritator(slices):
    if type(slices) == slice:
        if slices.start == slices.stop == slices.step == None:
            return ':'
        elif slices.start == None and slices.stop != None:
            return f' : {slices.stop}'
        elif slices.start != None and slices.stop == None:
            return f'{slices.start} : '
        else:
            return f'{slices.start} : {slices.stop}'
    else:
        return slices

In [10]:
list1 = [[1,2,3], [4,5,6], [7,8,9]]
a = Array(list1)
b = np.array(list1)
c = list1

In [11]:
b

array([[1, 2, 3],
       [4, 5, 6],
       [7, 8, 9]])

In [12]:
slicesings = [[slice(0, 2), 1], [1, slice(0, 2)], [0, 1], [1, 0], [1, slice(None)],[slice(None), 1],\
              [1,slice(1, None)],[slice(1, None), 1]]
cool = True
for en, sl in enumerate(slicesings):
    print(f'array[{slice_intepritator(sl[0])}, {slice_intepritator(sl[1])}]', 'np:', b[sl[0], sl[1]], 'my:', a[sl[0], sl[1]])
    if np.all(b[sl[0], sl[1]] == a[sl[0], sl[1]]):
        print(en + 1, 'Ok')
    else:
        cool = False
        print(en + 1, 'Ko', [sl[0], sl[1]])
        
    print(f'array[{slice_intepritator(sl[0])}][{slice_intepritator(sl[1])}]','np:', b[sl[0]][sl[1]], 'my:', a[sl[0]][sl[1]])
    
    if np.all(b[sl[0]][sl[1]] == a[sl[0]][sl[1]]):
        print(en + 1, 'Ok', '\n')
    else:
        cool = False
        print(en + 1, 'Ko', [sl[0]][sl[1]], '\n')
print('--------------------')
if cool:
    print('Ошибок нет')
else:
    print('Ошибка!!!!!')

array[0 : 2, 1] np: [2 5] my: [2, 5]
1 Ok
array[0 : 2][1] np: [4 5 6] my: [4, 5, 6]
1 Ok 

array[1, 0 : 2] np: [4 5] my: [4, 5]
2 Ok
array[1][0 : 2] np: [4 5] my: [4, 5]
2 Ok 

array[0, 1] np: 2 my: 2
3 Ok
array[0][1] np: 2 my: 2
3 Ok 

array[1, 0] np: 4 my: 4
4 Ok
array[1][0] np: 4 my: 4
4 Ok 

array[1, :] np: [4 5 6] my: [4, 5, 6]
5 Ok
array[1][:] np: [4 5 6] my: [4, 5, 6]
5 Ok 

array[:, 1] np: [2 5 8] my: [2, 5, 8]
6 Ok
array[:][1] np: [4 5 6] my: [4, 5, 6]
6 Ok 

array[1, 1 : ] np: [5 6] my: [5, 6]
7 Ok
array[1][1 : ] np: [5 6] my: [5, 6]
7 Ok 

array[1 : , 1] np: [5 8] my: [5, 8]
8 Ok
array[1 : ][1] np: [7 8 9] my: [7, 8, 9]
8 Ok 

--------------------
Ошибок нет


## Тестирование ошибок

In [32]:
a = Array([1,2,3])
b = Array([4,5,6,7])
print(a + b)

ValueError: Сложение векторов разной длины недопустимо

In [33]:
print(a + dict)

TypeError: Вектор нельзя сложить с <class 'dict'>

In [34]:
print(a * dict)

TypeError: Вектор нельзя умножить с <class 'dict'>

In [35]:
n = np.array([1,2,3])

In [36]:
a == n

array([ True,  True,  True])

In [37]:
np.array([1,2,3]) * np.array([1,2,3, 4])

ValueError: operands could not be broadcast together with shapes (3,) (4,) 

In [38]:
np.array([1,2,3]) * dict

TypeError: unsupported operand type(s) for *: 'int' and 'type'

In [449]:
list1 = [[1,2,3], [4,5,6], [7,8,9]]
a = Array(list1)
b = np.array(list1)

In [454]:
a[1,1,1]

IndexError: Слишком много индексов для массива этого пространства

In [453]:
b[1,1,1]

IndexError: too many indices for array: array is 2-dimensional, but 3 were indexed